In [4]:
%load_ext autoreload

In [28]:
%autoreload
import mpmath as mp
import darkhistory.physics as phys

In [44]:
def F_n(n, epsmin, epsmax, T, tol=1e-3):
    F_sum = 0
    rel_err = tol*10
    k = 1
    while rel_err > tol:
        next_term1 = mp.gammainc(n+1, k*epsmin/T)/k**(n+1)
        next_term2 = mp.gammainc(n+1, k*epsmax/T)/k**(n+1)
        next_term = next_term1 - next_term2
        F_sum += next_term
        k += 1
        rel_err = next_term/F_sum
    return F_sum*T**(n+1)

In [49]:
def G(epsmin, epsmax, T, tol=1e-3):
    G_sum = mp.log(T)*F_n(0,epsmin,epsmax,T,tol)
    rel_err = tol*10
    k = 1
    while rel_err > tol:
        next_term1 = mp.exp(-k*epsmin/T)*mp.log(epsmin/T)/k + mp.e1(k*epsmin/T)
        next_term2 = mp.exp(-k*epsmax/T)*mp.log(epsmax/T)/k + mp.e1(k*epsmax/T)
        next_term = T*(next_term1 - next_term2)
        G_sum += next_term
        k += 1
        rel_err = next_term/G_sum
    return G_sum
        

In [66]:
Ee = 1e3+phys.me
Ep = 1e-3
T = 1000

gamma = Ee/phys.me
beta = mp.sqrt(1-1/gamma**2)

prefac = phys.c*phys.thomson_xsec*(3/16)/(gamma**3*beta**2)*(8*mp.pi)/(phys.ele_compton**3*phys.me**3)
epsmin = (1-beta)/(1+beta)*Ep
epsmax = Ep

term1 = (1+beta**2)/beta**2*mp.sqrt((1+beta)/(1-beta))*F_n(1,epsmin,epsmax,T)
term2 = (2/beta)*mp.sqrt((1-beta)/(1+beta))*Ep*F_n(0,epsmin,epsmax,T)
term3 = -(1-beta)**2/beta**2*mp.sqrt((1+beta)/(1-beta))*Ep**2*F_n(-1,epsmin,epsmax,T)
term4 = 2/(gamma*beta**2)*Ep*mp.log((1-beta)/(1+beta)*Ep)*F_n(0,epsmin,epsmax,T)
term5 = -(2/(gamma*beta**2))*Ep*G(epsmin,epsmax,T)
print(term1)
print(term2)
print(term3)
print(term4)
print(term5)
totallow = term1 + term2 + term3 + term4 + term5
print(totallow)

0.0302956947307968
0.00353486950028487
-0.0299819316830694
-0.422817908529274
-0.414462064390448
-0.83343134037171


In [67]:
epsmin = Ep
epsmax = (1+beta)/(1-beta)*Ep
term6 = -(1+beta**2)/beta**2*mp.sqrt((1-beta)/(1+beta))*F_n(1,epsmin,epsmax,T)
term7 = (2/beta)*mp.sqrt((1+beta)/(1-beta))*Ep*F_n(0,epsmin,epsmax,T)
term8 = (1+beta)/(gamma*beta**2)*Ep**2*F_n(-1,epsmin,epsmax,T)
term9 = -2/(gamma*beta**2)*Ep*mp.log((1+beta)/(1-beta)*Ep)*F_n(0,epsmin,epsmax,T)
term10 = (2/(gamma*beta**2))*Ep*G(epsmin,epsmax,T)

print(term6)
print(term7)
print(term8)
print(term9)
print(term10)
totalhigh = term6 + term7 + term8 + term9 + term10
print(totalhigh)





-0.0343308618058482
0.00453950320962774
0.0339753146366711
0.462088361592742
0.469673933446414
0.935946251079606


In [68]:
print((totallow+totalhigh)*prefac)

1.28779953807039
